<a href="https://colab.research.google.com/github/BLOSSOM1994/calibration_wand/blob/main/MyWork/find_P_and_Error_camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from PIL import Image
import numpy as np
import pandas as pd

In [2]:

def Normalization(nd, x):
    '''
    Normalization of coordinates (centroid to the origin and mean distance of sqrt(2 or 3).
    Input
    -----
    nd: number of dimensions, 3 here
    x: the data to be normalized (directions at different columns and points at rows)
    Output
    ------
    Tr: the transformation matrix (translation plus scaling)
    x: the transformed data
    '''

    x = np.asarray(x)
    m, s = np.mean(x, 0), np.std(x)
    if nd == 2:
        Tr = np.array([[s, 0, m[0]], [0, s, m[1]], [0, 0, 1]])
    else:
        Tr = np.array([[s, 0, 0, m[0]], [0, s, 0, m[1]], [0, 0, s, m[2]], [0, 0, 0, 1]])
        
    Tr = np.linalg.inv(Tr)
    x = np.dot( Tr, np.concatenate( (x.T, np.ones((1,x.shape[0]))) ) )
    x = x[0:nd, :].T

    return Tr, x



In [3]:

def DLTcalib(nd, xyz, uv):
    '''
    Camera calibration by DLT using known object points and their image points.
    Input
    -----
    nd: dimensions of the object space, 3 here.
    xyz: coordinates in the object 3D space.
    uv: coordinates in the image 2D space.
    The coordinates (x,y,z and u,v) are given as columns and the different points as rows.
    There must be at least 6 calibration points for the 3D DLT.
    Output
    ------
     L: array of 11 parameters of the calibration matrix.
     err: error of the DLT (mean residual of the DLT transformation in units of camera coordinates).
    '''
    if (nd != 3):
        raise ValueError('%dD DLT unsupported.' %(nd))
    
    # Converting all variables to numpy array
    xyz = np.asarray(xyz)
    uv = np.asarray(uv)

    n = xyz.shape[0]

    # Validating the parameters:
    if uv.shape[0] != n:
        raise ValueError('Object (%d points) and image (%d points) have different number of points.' %(n, uv.shape[0]))

    if (xyz.shape[1] != 3):
        raise ValueError('Incorrect number of coordinates (%d) for %dD DLT (it should be %d).' %(xyz.shape[1],nd,nd))

    if (n < 6):
        raise ValueError('%dD DLT requires at least %d calibration points. Only %d points were entered.' %(nd, 2*nd, n))
        
    # Normalize the data to improve the DLT quality (DLT is dependent of the system of coordinates).
    # This is relevant when there is a considerable perspective distortion.
    # Normalization: mean position at origin and mean distance equals to 1 at each direction.
    Txyz, xyzn = Normalization(nd, xyz)
    Tuv, uvn = Normalization(2, uv)

    A = []

    for i in range(n):
        x, y, z = xyzn[i, 0], xyzn[i, 1], xyzn[i, 2]
        u, v = uvn[i, 0], uvn[i, 1]
        A.append( [x, y, z, 1, 0, 0, 0, 0, -u * x, -u * y, -u * z, -u] )
        A.append( [0, 0, 0, 0, x, y, z, 1, -v * x, -v * y, -v * z, -v] )

    # Convert A to array
    A = np.asarray(A) 

    # Find the 11 parameters:
    U, S, V = np.linalg.svd(A)

    # The parameters are in the last line of Vh and normalize them
    L = V[-1, :] / V[-1, -1]
    print("\nL: ",L)
    # Camera projection matrix
    H = L.reshape(3, nd + 1)
    print("\nH: ",H)

    # Denormalization
    # pinv: Moore-Penrose pseudo-inverse of a matrix, generalized inverse of a matrix using its SVD
    H = np.dot( np.dot( np.linalg.pinv(Tuv), H ), Txyz )
    print("\nH: ",H)
    H = H / H[-1, -1]
    print("\nH:",H)
    L = H.flatten('A')
    print(L)

    # Mean error of the DLT (mean residual of the DLT transformation in units of camera coordinates):
    uv2 = np.dot( H, np.concatenate( (xyz.T, np.ones((1, xyz.shape[0]))) ) ) 
    uv2 = uv2 / uv2[2, :] 
    # Mean distance:
    err = np.sqrt( np.mean(np.sum( (uv2[0:2, :].T - uv)**2, 1)) ) 

    return L, err


In [4]:

def DLT(real,cam):
    xyz=real
    uv=cam
    # Known 3D coordinates
   # xyz = [[-875, 0, 9.755], [442, 0, 9.755], [1921, 0, 9.755], [2951, 0.5, 9.755], [-4132, 0.5, 23.618],
  #  [-876, 0, 23.618]]
    # Known pixel coordinates
   # uv = [[76, 706], [702, 706], [1440, 706], [1867, 706], [264, 523], [625, 523]]

    nd = 3
    P, err = DLTcalib(nd, xyz, uv)
    print('Matrix')
    print(P)
    print('\nError')
    print(err)
    return P,err


In [5]:
def camera (image_points):
  y=0
  cam_img=[]
  for line in image_points:
    arr=[]
    image_coord= line.split()
  #print(image_coord)
    i=0
    for x in image_coord:
      arr.append(x)
   # print(x)
      i=i+1
    cam_img.append(arr)
    y=y+1
  cam_img=np.array(cam_img).astype(np.float)
  return cam_img

In [6]:
def world(world_point):
  y=0
  real_img=[]
  for line in world_point:
   arr=[]
   world_coord= line.split()
  #print(image_coord)
   i=0
   for x in world_coord:
     arr.append(x)
   # print(x)
     i=i+1
   arr.pop(3)
   real_img.append(arr)
   y=y+1
  real_img=np.array(real_img).astype(np.float)
  return real_img

In [7]:
CameraName=['camera11.txt','camera12.txt','camera21.txt','camera22.txt','camera31.txt','camera32.txt']
WorldName=['mokhtasatvaghei1.txt','mokhtasatvaghei2.txt']

In [8]:
if __name__ == "__main__":
  i_im=0
  j_im=0
  ii=1
  while i_im < len(CameraName) :
      image_points=open(CameraName[i_im],"r")
      cam_img=camera(image_points)
      if j_im ==2:
        j_im=0
        ii=ii+1
      world_point=open(WorldName[j_im],"r")
      real_img=world(world_point)
      P,err=DLT(real_img,cam_img)
      #save P and error
      file_name_img= "cam"+str(ii)+"_point"+str(j_im+1)+".txt"
      file_img = open(file_name_img,"w+")
      file_img.write(   '''
      Normalization of coordinates (centroid to the origin and mean distance of sqrt(2 or 3).

      Input
      -----
      nd: number of dimensions, 3 here
      x: the data to be normalized (directions at different columns and points at rows)
    
      Output
      ------
      Tr: the transformation matrix (translation plus scaling)
      x: the transformed data

      \n Matrix P:\n ''')
      file_img.write(str(P))
      file_img.write("\n\n Error: ")
      file_img.write(str(err))
      file_img.close()
      j_im=j_im+1
      i_im=i_im+1
      image_points.close()
      world_point.close()


L:  [ 4.85809822e+00 -1.22973654e-01 -7.40600450e-01 -2.16687547e-02
 -1.11923219e-01  5.05250029e+00 -5.64208239e-01 -4.65125208e-03
  1.06828996e-01  7.00913198e-02  3.84364488e-01  1.00000000e+00]

H:  [[ 4.85809822e+00 -1.22973654e-01 -7.40600450e-01 -2.16687547e-02]
 [-1.11923219e-01  5.05250029e+00 -5.64208239e-01 -4.65125208e-03]
 [ 1.06828996e-01  7.00913198e-02  3.84364488e-01  1.00000000e+00]]

H:  [[7.39039643e-01 2.69519646e-02 1.38634935e-01 9.37977571e+00]
 [3.55133709e-02 7.32384529e-01 1.05432235e-01 5.25640897e+00]
 [1.05509317e-04 6.92254686e-05 3.79616362e-04 1.81504175e-02]]

H: [[4.07175009e+01 1.48492257e+00 7.63811271e+00 5.16780162e+02]
 [1.95661455e+00 4.03508365e+01 5.80880491e+00 2.89602648e+02]
 [5.81305181e-03 3.81398767e-03 2.09150209e-02 1.00000000e+00]]
[4.07175009e+01 1.48492257e+00 7.63811271e+00 5.16780162e+02
 1.95661455e+00 4.03508365e+01 5.80880491e+00 2.89602648e+02
 5.81305181e-03 3.81398767e-03 2.09150209e-02 1.00000000e+00]
Matrix
[4.07175009e